In [1]:
with open("key.txt", "r") as file:
    API_KEY_CLARIN = file.read().strip()

MODELS_ENDPOINT = "https://services.clarin-pl.eu/api/v1/oapi/models"
COMPLETIONS_ENDPOINT = "https://services.clarin-pl.eu/api/v1/oapi/chat/completions"
CLARIN_BASE_URL = "https://services.clarin-pl.eu/api/v1/oapi"

MODEL_ID = "bielik"

In [2]:
import requests
import pandas as pd
import json
import time
import requests

In [3]:
def prompt_chat_default_settings(model_id, prompt):
    url = COMPLETIONS_ENDPOINT
    headers = {"Authorization": f"Bearer {API_KEY_CLARIN}", "Content-Type": "application/json"}
    messages = []
    messages.append({"role": "user", "content": prompt})
    data = {
        "model": model_id,
        "messages": messages,
        #"max_tokens": max_tokens
    }
    response = requests.post(url, json=data, headers=headers)
    if response.status_code == 200:
        try:
            data = response.json()
            if isinstance(data, dict) and "choices" in data and data["choices"]:
                return data["choices"][0].get("message", {}).get("content", None)
            else:
                return None  # Handle unexpected response structure
        except (ValueError, TypeError, AttributeError):
            return None  # Handle cases where response.json() is None or not as expected
    else:
        return None  # Handle non-200 status codes

In [4]:
def prompt_chat_custom_temperature(model_id, prompt, temperature = 0.3):
    url = COMPLETIONS_ENDPOINT
    headers = {"Authorization": f"Bearer {API_KEY_CLARIN}", "Content-Type": "application/json"}
    messages = []
    messages.append({"role": "user", "content": prompt})
    data = {
        "model": model_id,
        "messages": messages,
        "temperature": temperature
        #"max_tokens": max_tokens
    }
    response = requests.post(url, json=data, headers=headers)
    if response.status_code == 200:
        try:
            data = response.json()
            if isinstance(data, dict) and "choices" in data and data["choices"]:
                return data["choices"][0].get("message", {}).get("content", None)
            else:
                return None  # Handle unexpected response structure
        except (ValueError, TypeError, AttributeError):
            return None  # Handle cases where response.json() is None or not as expected
    else:
        return None  # Handle non-200 status codes

In [5]:
df = pd.read_csv("data/dataset.csv", delimiter="|")
df

,en_text,pl_text
0,"Hey, everyone! Wanna hear about Fyresdal?","Ej, ludzie! Chcecie posłuchać o Fyresdal?"
1,"Hey, good folk! Wanna hear what our brave guar...","Ej, ludzie! Chcecie usłyszeć, co nasz dzielny ..."
2,"Shut your trap, dammit!","Stul pysk, do cholery!"
3,Four against one - and you came out alive.,Czterech na jednego. A dałeś radę.
4,Shame to let such a lad waste away behind bars...,"Szkoda, żeby taki chłop marniał za kratkami......"
...,...,...
8375,Farewell.,Bywaj.
8376,Farewell.,Bywaj.
8377,What do you have on offer?,Pokaż mi swoje towary.
8378,Show me what you have in stock.,Pokaż mi swoje towary.


In [6]:
sentences = ["Unless… She coexists with drowners… Maybe I could make an extract from their glands, cover up my scent.", 
             "When'd you lose your voice? Can you show me, hold up some fingers?",
             "Pirate Leader",
             "You're dull as dirt. *Yaaawn*",
             "Jarl of Undvik - tell me about his death.",
             "Let's go see Udalryk. Be interesting to see if there actually is a ghost in all this.",
             "Bad way to play it out.",
             "So? Ready?",
             "It'll cost you. Scratch my back, I'll scratch yours."]

In [7]:
prompt_chat_custom_temperature(
    MODEL_ID, 
    f"Przetłumacz na polski i nie dodawaj żadnych dodatkowych wyjaśnień, napisz tylko samo tłumaczenie: '{sentences[3]}'", 
    0.0
)

'Jesteś nudny jak flaki z olejem. *Ziewnięcie*'

In [8]:
context = "Zdanie jest elementem dialogu. Wypowiada je mężczyzna w stronę kobiety. \n"

In [9]:
prompt_chat_custom_temperature(
    MODEL_ID, 
    f"Oto kontekst: {context} Przetłumacz na polski i nie dodawaj żadnych dodatkowych wyjaśnień, napisz tylko samo tłumaczenie: '{sentences[3]}'", 
    0.0
)

'Jesteś nudna jak flaki z olejem. *Ziewnięcie*'

In [10]:
rule1 = "W dialogu mogą wystąpić słowa opisujące dzwięki wydawane przez osobę, która mówi. Podczas tłumaczenia zamień te słowa na faktycznie dzwięki, wydawane podczas tej czynności. \n" 
examples = "Yawn - Aaah \n Sneeze - Apsik \n"

In [11]:
prompt_chat_custom_temperature(
    MODEL_ID, 
    f"Zasada: {rule1} Przykłady: {examples} Przetłumacz na polski i nie dodawaj żadnych dodatkowych wyjaśnień, napisz tylko samo tłumaczenie: '{sentences[3]}'", 
    0.0
)

'Jesteś nudny jak flaki z olejem. *Aaah*'

In [12]:
prompt_chat_custom_temperature(
    MODEL_ID, 
    f"Oto kontekst: {context} Zasada: {rule1} Przykłady: {examples} Przetłumacz na polski i nie dodawaj żadnych dodatkowych wyjaśnień, napisz tylko samo tłumaczenie: '{sentences[3]}'", 
    0.0
)

'Jesteś nudny jak flaki z olejem. *Aaah*'

In [13]:
prompt_chat_custom_temperature(
    MODEL_ID, 
    f"Oto podane zdanie: {sentences[3]} \n Kontekst: {context} Zasada: {rule1} Przykłady: {examples} Przetłumacz podane zdanie na polski i nie dodawaj żadnych dodatkowych wyjaśnień, napisz tylko samo tłumaczenie. Podczas tłumaczenia uwzględnij zasady i kontekst:", 
    0.0
)

'Jesteś nudna jak flaki z olejem. *Ziewnięcie*'

In [14]:
translationv1 = prompt_chat_custom_temperature(
    MODEL_ID, 
    f"Przetłumacz na polski i nie dodawaj żadnych dodatkowych wyjaśnień, napisz tylko samo tłumaczenie: '{sentences[3]}'", 
    0.0
)
translationv1

'Jesteś nudny jak flaki z olejem. *Ziewnięcie*'

In [15]:
translationv2 = prompt_chat_custom_temperature(
    MODEL_ID, 
    f"Oto kontekst: {context} Przepisz podane zdanie na polski zgodnie z podanym kotekstem i nie dodawaj żadnych dodatkowych wyjaśnień, napisz tylko samo tłumaczenie: '{translationv1}'",
    0.0
)
translationv2

'"Jesteś tak nudny, że mogłabym zasnąć stojąc." *Ziewnięcie*'

In [16]:
context2 = "Zdanie wypowiadane jest przez mężczyzne w stronę kobiety \n"

In [17]:
translationv2 = prompt_chat_custom_temperature(
    MODEL_ID, 
    f"Zdanie po tłumaczniu bez kontekstu: '{translationv1}'\n Kontekst zdania: {context}\n Odmień wyrazy w podanym tłumaczeniu tak aby pasowały do kontekstu zdania. Napisz tylko i wyłącznie samo poprawione tłumaczenie i nie dodawaj żadnych dodatkowych wyujaśnień:",
    0.0
)
translationv2

'Jesteś nudna jak flaki z olejem. *Ziewnięcie*'

In [18]:
translationv3 = prompt_chat_custom_temperature(
    MODEL_ID, 
    f"Zdanie po tłumaczniu '{translationv2}'\n Zasady: {rule1}\n Przykłady zasady: {examples}\n Zastosuj zasady wedle przykładów na podanym zdaniu i napisz tylko i wyłącznie samo poprawione zdanie, nie dodawaj żadnych dodatkowych wyjaśnień:",
    0.0
)
translationv3

'Jesteś nudna jak flaki z olejem. *Aaah*'

In [19]:
prompt_chat_default_settings(
    MODEL_ID, 
    "Opis relację pomiędzy wyrazami: Yawn - Aaah, Bark bark - hał hał\n"
)

'Relacja pomiędzy wyrazami "yawn" i "Aaah" oraz "bark bark" i "hał hał" jest podobna. Obie pary wyrazów odnoszą się do dźwięków lub odgłosów wydawanych przez ludzi lub inne istoty. \n\n"Yawn" oznacza działanie ziewania (westchnienia) połączone z wydaniem charakterystycznego dźwięku. Dlatego też słowo to jest powiązane z dźwiękiem "Aaah", który najlepiej oddaje brzmienie tego gestu. Przykład zdania: "He yawned and said, \'Aaah!\'"\n\nPodobnie "bark bark" to odtworzenie odgłosu wydobywającego się z ust (ust lub nosa), często w akcie ekspresji (np. żalu, zaskoczenia, niepewności itd.). Wyrażenie "aha" jest zwykłą konwencjonalną frazą używaną dla oddania takiego typu dźwięku, np.: "She spoke with a barking voice and said, \'Aha, I don\'t like this.\'"\n\nOba przykłady pokazują powiązanie dźwiękowych gestów werbalnych ("yawn" i "bark") ze słowem lub frazą („\xa0aha\xa0”, "aha"), które są używane jako zamiennik lub imitacja tychże. Wyrażają one ideę wydawania dźwięków za ich pomocą.'

In [20]:
prompt_chat_custom_temperature(MODEL_ID,
    f"Oto przykładowe zdanie: {sentences[3]}. Jakie polecenie napisałbyś do modelu, aby zamienić w nim wszystkie czynności opisujące wydawany dzwięk na faktycznie wydawane dzwięki? \n",
    0.3
)

'Aby zamienić wszystkie czynności opisujące wydawany dźwięk na faktyczne dźwięki w podanym zdaniu "You\'re dull as dirt. *Yaaawn*", możesz użyć następującego polecenia dla modelu językowego:\n\n"Zamień wszystkie wyrażenia opisujące wydawanie dźwięków na same dźwięki w tym zdaniu."\n\nTo polecenie poprosi model o zidentyfikowanie i zamianę takich wyrażeń jak "you\'re yawning" lub "you yawned" na samo "yawn". W przypadku tego konkretnego zdania, nie ma potrzeby dokonywania żadnych zmian, ponieważ zawiera ono już faktyczny dźwięk (*Yaaawn*). Jednak to polecenie będzie skuteczne dla innych zdań, które mogą zawierać opisy dźwięków zamiast samych dźwięków.'

In [21]:
translationv2

'Jesteś nudna jak flaki z olejem. *Ziewnięcie*'

In [22]:
prompt_chat_custom_temperature(MODEL_ID,
    f"Zidentyfikuj wszystkie wyrazy opisujące wydawane dzwięki w zdaniu i je wypisz: {translationv2}\n",
    0.0
)

'W podanym zdaniu "Jesteś nudna jak flaki z olejem. *Ziewnięcie*" znajduje się jeden wyraz opisujący wydawany dźwięk:\n\n- Ziewnięcie\n\nTo słowo odnosi się do charakterystycznego dźwięku, jaki wydaje osoba ziewająca. W tym kontekście jest ono użyte jako element opisu sytuacji lub stanu emocjonalnego, a nie bezpośrednio jako część zdania.'

In [23]:
answer = "'W podanym zdaniu Jesteś nudna jak flaki z olejem. *Ziewnięcie* znajduje się jeden wyraz opisujący wydawany dźwięk:\n\n- Ziewnięcie\n\nTo słowo odnosi się do charakterystycznego dźwięku, jaki wydaje osoba ziewająca. W tym kontekście jest ono użyte jako element opisu sytuacji lub stanu emocjonalnego, a nie bezpośrednio jako część zdania.'"

In [24]:
prompt_chat_custom_temperature(MODEL_ID,
    f"Zdanie: {translationv2}\n"+
    "Zastąp wszystkie opisy dźwięków wydawanych podczas czynności, odpowiednikami dźwiękonaśladowczymi, które fonetycznie oddają realny dźwięk tych czynności – tak, aby czytając je, można było niemal usłyszeć ten dźwięk",
    0.3
)

'Oto zdanie zastąpione dźwiękonaśladowczymi odpowiednikami dźwięków:\n\nJesteś nudna jak flaki z olejem. *Brrrum brrum*\n\nW tym przypadku "ziewnięcie" zostało zastąpione przez "brrrum brrum", co fonetycznie oddaje dźwięk ziewania. Dźwięk ten jest często opisywany jako głębokie, powolne i rytmiczne "brum brum".'

In [25]:
prompt_chat_custom_temperature(
    MODEL_ID, 
    f"Znajdź w podanym zdaniu powiedzenia, kórych znaczenie jest inne niż dosłowne znaczenia słów, a następnie przetłumacz je na odpowiedniki po polsku: '{sentences[-1]}'", 
    0.0
)

'W podanych zdaniach występują dwa idiomatyczne wyrażenia:\n\n1. "It\'ll cost you." - To wyrażenie oznacza, że coś będzie miało dla kogoś negatywne konsekwencje lub spowoduje problemy. W języku polskim można to oddać jako "To się zemści" lub "To się obróci przeciwko tobie".\n\n2. "Scratch my back, I\'ll scratch yours." - To popularne angielskie powiedzenie oznaczające propozycję wzajemnej pomocy lub współpracy. W języku polskim można to przetłumaczyć jako "Jak ty mi tak ja tobie" lub "Ręka rękę myje".\n\nPodsumowując:\n- "It\'ll cost you." = "To się zemści" / "To się obróci przeciwko tobie"\n- "Scratch my back, I\'ll scratch yours." = "Jak ty mi tak ja tobie" / "Ręka rękę myje"'

In [28]:
prompt_chat_custom_temperature(
    MODEL_ID, 
    f"Znajdź w podanym zdaniu powiedzenia/idiomy, kórych znaczenie jest inne niż dosłowne znaczenia słów, a następnie przetłumacz je na odpowiedniki po polsku, wypisz po 3 sposoby tlumaczenia na każdy idiom: '{sentences[-1]}'", 
    0.0
)

'W podanych zdaniach zidentyfikowałem dwa idiomy:\n\n1. "It\'ll cost you."\n   - Dosłowne znaczenie: Będzie cię to kosztować.\n   - Znaczenie przenośne: Oznacza, że coś będzie miało dla ciebie negatywne konsekwencje lub skutki uboczne, często nieoczekiwane lub niepożądane.\n   \n   Trzy sposoby tłumaczenia na język polski:\n   a) To się na tobie odbije.\n   b) To cię uderzy rykoszetem.\n   c) To cię dopadnie.\n\n2. "Scratch my back, I\'ll scratch yours."\n   - Dosłowne znaczenie: Podrap mnie po plecach, ja podrapię ciebie.\n   - Znaczenie przenośne: Oznacza propozycję wzajemnej pomocy lub współpracy, gdzie obie strony mają korzyści ze swoich działań.\n\n   Trzy sposoby tłumaczenia na język polski:\n   a) Ja ci pomogę, ty mi pomożesz.\n   b) Wzajemna przysługa.\n   c) Ktoś za kogoś.\n\nWarto zauważyć, że te idiomy są powszechnie używane w języku angielskim i ich dokładne tłumaczenie może nie oddawać pełnego znaczenia w języku polskim, ale przedstawione przykłady starają się jak najlepie

In [31]:
prompt_chat_custom_temperature(
    MODEL_ID, 
    f"Znajdź w podanym zdaniu powiedzenia/idiomy, kórych znaczenie jest inne niż dosłowne znaczenia słów, a następnie przetłumacz je na odpowiedniki po polsku '{sentences[-1]}' i wypisz je.\n"+
    f"Następnie korzystając ze znalezionych odpowiedników idiomów, przetłumacz całe zdanie na język polski: '{sentences[-1]}'", 
    0.0
)

'W podanych zdaniach zidentyfikowałem dwa idiomy:\n\n1. "It\'ll cost you" - oznacza to, że coś będzie miało dla kogoś negatywne konsekwencje lub spowoduje problemy. W języku polskim odpowiednikiem tego idiomu jest "To się na Tobie odbije".\n\n2. "Scratch my back, I\'ll scratch yours" - to wyrażenie oznacza propozycję wzajemnej pomocy lub współpracy. W języku polskim można to oddać jako "Jak ty mi tak ja tobie".\n\nTeraz przetłumaczę całe zdanie na język polski:\n\n"To się na Tobie odbije. Jak ty mi tak ja tobie."\n\nTo zdanie sugeruje, że ktoś ostrzega drugą osobę przed potencjalnymi negatywnymi skutkami jej działań oraz proponuje wzajemną pomoc lub współpracę.'

In [33]:
zdanie = 'W podanych zdaniach zidentyfikowałem dwa idiomy:\n\n1. "It\'ll cost you" - oznacza to, że coś będzie miało dla kogoś negatywne konsekwencje lub spowoduje problemy. W języku polskim odpowiednikiem tego idiomu jest "To się na Tobie odbije".\n\n2. "Scratch my back, I\'ll scratch yours" - to wyrażenie oznacza propozycję wzajemnej pomocy lub współpracy. W języku polskim można to oddać jako "Jak ty mi tak ja tobie".\n\nTeraz przetłumaczę całe zdanie na język polski:\n\n"To się na Tobie odbije. Jak ty mi tak ja tobie."\n\nTo zdanie sugeruje, że ktoś ostrzega drugą osobę przed potencjalnymi negatywnymi skutkami jej działań oraz proponuje wzajemną pomoc lub współpracę.'

In [36]:
prompt_chat_custom_temperature(
    MODEL_ID, 
    f"Oto próba przetłumaczenia zdania przez model: {zdanie} \n"+
    f"Wyciągnij z podanego tekstu samo tłumaczenie zdania i nie dodawaj żadnych dodatkowych wyjaśnień, napisz tylko samo tłumaczenie:",
    0.0
)

'To się na Tobie odbije. Jak ty mi tak ja tobie.'

In [45]:
sentence_to_translate = "You're funny Jane but it'll cost you. Scratch my back, I'll scratch yours."
context = "Zdanie jest elementem dialogu. Wypowiada je mężczyzna w stronę kobiety. \n"

In [46]:
output1 = prompt_chat_custom_temperature(
    MODEL_ID,
    f"Zdanie do przetłumaczenia: {sentence_to_translate} \n"+
    f"Kontekst: {context} \n"+
    f"Znajdź w podanym do tłumaczenia zdaniu powiedzenia/idiomy, kórych znaczenie jest inne niż dosłowne znaczenia słów, a następnie przetłumacz je na odpowiedniki po polsku i wypisz je.\n"+ 
    f"Następnie korzystając ze znalezionych odpowiedników idiomów, przetłumacz całe zdanie na język polski. \n"+
    f"Następnie przepisz podane zdanie na polski zgodnie z podanym kotekstem",
    0.0
)
output1

'W podanym zdaniu możemy zidentyfikować dwa idiomy:\n\n1. "It\'ll cost you" - oznacza to, że coś będzie mieć negatywne konsekwencje lub spowoduje problemy dla osoby, która się tego podejmie.\n   Odpowiednik po polsku: "To cię będzie kosztować"\n\n2. "Scratch my back, I\'ll scratch yours" - to popularne powiedzenie oznaczające wzajemną pomoc lub współpracę między dwiema osobami.\n   Odpowiednik po polsku: "Jak ty mnie, tak ja tobie"\n\nTeraz przetłumaczmy całe zdanie na język polski, uwzględniając te idiomy:\n\n"You\'re funny Jane but it\'ll cost you."\n"Jesteś zabawna, Janie, ale to cię będzie kosztować."\n\n"Scratch my back, I\'ll scratch yours."\n"Jak ty mnie, tak ja tobie."\n\nBiorąc pod uwagę kontekst dialogu, gdzie mężczyzna wypowiada te słowa do kobiety, możemy przepisać to zdanie na polski w następujący sposób:\n\n"Jesteś zabawna, Janie, ale to cię będzie kosztować. Jak ty mnie, tak ja tobie."\n\nTo zdanie sugeruje, że mężczyzna docenia poczucie humoru kobiety, ale jednocześnie 

In [47]:
output2 = prompt_chat_custom_temperature(
    MODEL_ID, 
    f"Oto próba przetłumaczenia zdania: {output1} \n"+
    f"Wyciągnij z podanego tekstu samo tłumaczenie zdania i nie dodawaj żadnych dodatkowych wyjaśnień, napisz tylko samo tłumaczenie:",
    0.0
)
output2

'"Jesteś zabawna, Janie, ale to cię będzie kosztować. Jak ty mnie, tak ja tobie."'

In [48]:
context = "Zdanie jest elementem dialogu. Wypowiada je mężczyzna w stronę mężczyzny. \n"

In [49]:
output1 = prompt_chat_custom_temperature(
    MODEL_ID,
    f"Zdanie do przetłumaczenia: {sentence_to_translate} \n"+
    f"Kontekst: {context} \n"+
    f"Znajdź w podanym do tłumaczenia zdaniu powiedzenia/idiomy, kórych znaczenie jest inne niż dosłowne znaczenia słów, a następnie przetłumacz je na odpowiedniki po polsku i wypisz je.\n"+ 
    f"Następnie korzystając ze znalezionych odpowiedników idiomów, przetłumacz całe zdanie na język polski. \n"+
    f"Następnie przepisz podane zdanie na polski zgodnie z podanym kotekstem",
    0.0
)
output2 = prompt_chat_custom_temperature(
    MODEL_ID, 
    f"Oto próba przetłumaczenia zdania: {output1} \n"+
    f"Wyciągnij z podanego tekstu samo tłumaczenie zdania i nie dodawaj żadnych dodatkowych wyjaśnień, napisz tylko samo tłumaczenie:",
    0.0
)
output2

'Jesteś zabawny, ale to ci się odbije czkawką. Jeśli ty mi pomożesz, ja też ci pomogę.'

In [52]:
names = prompt_chat_custom_temperature(
    MODEL_ID, 
    f"Wypisz wszystkie nazwy własne oraz imiona z podanego zdania: {sentence_to_translate} \n",
    0.0
)
names

'Oto lista wszystkich nazw własnych i imion z podanego zdania:\n\n1. Jane - imię osobowe\n2. Scratch - nie jest to ani imię, ani nazwa własna; jest to czasownik oznaczający "drapać"\n3. Back - również nie jest to ani imię, ani nazwa własna; jest to rzeczownik oznaczający "plecy"\n4. I\'ll - zaimek osobowy w drugiej osobie liczby pojedynczej\n5. Yours - zaimek dzierżawczy oznaczający "twoje"\n\nPodsumowując, jedyną nazwą własną lub imieniem w tym zdaniu jest "Jane".'

In [54]:
translation_with_names = prompt_chat_custom_temperature(
    MODEL_ID, 
    f"Zdanie do przetłumaczenia {sentence_to_translate} \n"+
    f"Oto znalezione nazwy własne w podanym do tłumaczenia zdaniu {names} \n"+
    f"Przetłumacz podane do tłumaczenia zdanie na język polski. Jeżeli nazwa własna nie jest po polsku, nie odmieniaj jej, natomiast znaleziona nazwa własne jest po Polsku, odmień ją. \n",
    0.0
)
translation_with_names

'Tłumaczenie podanego zdania na język polski:\n\n"Jesteś zabawna, Jane, ale będzie cię to kosztować. Podrap mnie po plecach, a ja podrapię ciebie."\n\nW tym tłumaczeniu:\n- "Jane" pozostaje bez zmian jako imię własne.\n- Czasowniki "scratch" (podrapać) i "back" (plecy) zostały przetłumaczone dosłownie.\n- Zaimki "I\'ll" (ja) i "yours" (twoje) zostały oddane odpowiednimi formami w języku polskim.'

In [57]:
output1 = prompt_chat_custom_temperature(
    MODEL_ID,
    f"Zdanie do przetłumaczenia: {sentence_to_translate} \n"+
    f"Kontekst: {context} \n"+
    f"Wypisz wszystkie nazwy własne oraz imiona z podanego zdania \n"+
    f"Znajdź w podanym do tłumaczenia zdaniu powiedzenia/idiomy, kórych znaczenie jest inne niż dosłowne znaczenia słów, a następnie przetłumacz je na odpowiedniki po polsku i wypisz je.\n"+ 
    f"Następnie korzystając ze znalezionych odpowiedników idiomów oraz nazw własnych, przetłumacz całe zdanie na język polski. Jeżeli nazwa własna nie jest po polsku, nie odmieniaj jej. \n"+
    f"Następnie przepisz podane zdanie na polski zgodnie z podanym kotekstem",
    0.0
)
output2 = prompt_chat_custom_temperature(
    MODEL_ID, 
    f"Oto próba przetłumaczenia zdania: {output1} \n"+
    f"Wyciągnij z podanego tekstu samo tłumaczenie zdania i nie dodawaj żadnych dodatkowych wyjaśnień, napisz tylko samo tłumaczenie:",
    0.0
)
output2

'"Jesteś zabawny Jane, ale to cię będzie kosztować. Podrap mnie po plecach, ja podrapię ciebie."'

In [58]:
context = "Zdanie jest elementem dialogu. Wypowiada je mężczyzna w stronę kobiety. \n"

In [59]:
output1 = prompt_chat_custom_temperature(
    MODEL_ID,
    f"Zdanie do przetłumaczenia: {sentence_to_translate} \n"+
    f"Kontekst: {context} \n"+
    f"Wypisz wszystkie nazwy własne oraz imiona z podanego zdania \n"+
    f"Znajdź w podanym do tłumaczenia zdaniu powiedzenia/idiomy, kórych znaczenie jest inne niż dosłowne znaczenia słów, a następnie przetłumacz je na odpowiedniki po polsku i wypisz je.\n"+ 
    f"Następnie korzystając ze znalezionych odpowiedników idiomów oraz nazw własnych, przetłumacz całe zdanie na język polski. Jeżeli nazwa własna nie jest po polsku, nie odmieniaj jej. \n"+
    f"Następnie przepisz podane zdanie na polski zgodnie z podanym kotekstem",
    0.0
)
output2 = prompt_chat_custom_temperature(
    MODEL_ID, 
    f"Oto próba przetłumaczenia zdania: {output1} \n"+
    f"Wyciągnij z podanego tekstu samo tłumaczenie zdania i nie dodawaj żadnych dodatkowych wyjaśnień, napisz tylko samo tłumaczenie:",
    0.0
)
output2

'"Jesteś zabawna, Jane, ale to cię będzie kosztować. Podrap mnie po plecach, ja podrapię ciebie."'

In [60]:
output1

'### Nazwy własne i imiona\nW podanym zdaniu występuje jedno imię:\n- Jane\n\n### Idiomy i ich odpowiedniki po polsku\n1. "You\'re funny Jane but it\'ll cost you." - Odpowiednik: "Jesteś zabawna, Jane, ale to cię będzie kosztować."\n   - To wyrażenie sugeruje, że osoba może być zabawna lub atrakcyjna, ale za pewną cenę lub konsekwencję.\n\n2. "Scratch my back, I\'ll scratch yours." - Odpowiednik: "Podrap mnie po plecach, ja podrapię ciebie."\n   - To popularne powiedzenie oznaczające wzajemną pomoc lub współpracę.\n\n### Tłumaczenie całego zdania na język polski\n"Jesteś zabawna, Jane, ale to cię będzie kosztować. Podrap mnie po plecach, ja podrapię ciebie."\n\n### Przepisane zdanie zgodnie z kontekstem\nMężczyzna wypowiadający te słowa do kobiety:\n\n"Jesteś zabawna, Jane, ale to cię będzie kosztować. Podrap mnie po plecach, ja podrapię ciebie."'

In [61]:
output1 = prompt_chat_custom_temperature(
    MODEL_ID,
    f"Zdanie do przetłumaczenia: {sentence_to_translate} \n"+
    f"Kontekst: {context} \n"+
    f"Znajdź w podanym do tłumaczenia zdaniu powiedzenia/idiomy, kórych znaczenie jest inne niż dosłowne znaczenia słów, a następnie przetłumacz je na odpowiedniki po polsku i wypisz je.\n"+ 
    f"Następnie korzystając ze znalezionych odpowiedników idiomów oraz nazw własnych, przetłumacz całe zdanie na język polski. \n"+
    f"Następnie przepisz podane zdanie na polski zgodnie z podanym kotekstem",
    0.0
)
output2 = prompt_chat_custom_temperature(
    MODEL_ID, 
    f"Oto próba przetłumaczenia zdania: {output1} \n"+
    f"Wyciągnij z podanego tekstu samo tłumaczenie zdania i nie dodawaj żadnych dodatkowych wyjaśnień, napisz tylko samo tłumaczenie:",
    0.0
)
output2

'Jesteś zabawna, Janino, ale to ci się odbije czkawką. Jak Kuba Bogu, tak Bóg Kubie.'